# Topic Modelling
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [7]:
import json
from pathlib import Path
import glob
from nltk.corpus import stopwords
from dateutil import parser

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [8]:
stop_words = stopwords.words('german')

## Load Data

In [6]:
records = []
for f in glob.glob(str(Path('../export/tags/*.json'))):
    records = records + json.load(open(f, 'r', encoding='utf-8'))

print(len(records))

KeyboardInterrupt: 

In [ ]:
# Members only, no Presidents
r_members = list(filter(lambda x: x['ismember'] == True, records))
r_members = list(filter(lambda x: x['funktion'] not in ['Präsidium', '2. Vizepräsidium', '1. Vizepräsidium'], r_members))
print(len(r_members))

## Filter by date if needed

In [ ]:
from_date = parser.parse("2016-05-01")
r_members = list(filter(lambda x: parser.parse(x['sitzung_date']) >= from_date, r_members))
print(len(r_members))

## Prepare Data

In [ ]:
# Remove Stopwords and only keep noun, adj, vb and adv

keep = ['NN', 'NE',
    'ADJA', 'ADJD',
    'VMFIN', 'VAFIN', 'VVFIN', 'VAIMP', 'VVIMP', 'VVINF', 'VAINF', 'VMINF', 'VVIZU', 'VVPP', 'VMPP', 'VAPP',
    'ADV']

for votum in r_members:
    # Reversed Loop, we will remove items from the list during loop!
    for i in reversed(range(0, len(votum['tags']))):
        tag = votum['tags'][i]
        if tag[0].lower() in stop_words:

            # Stopword found. Remove
            votum['tags'].remove(tag)

        elif tag[2] not in keep:
            
            # Not tag we need. Remove
            votum['tags'].remove(tag)

In [31]:
# Stick the words together to get a nice list for our corpora and split by sex
data_m = []
data_w = []
for votum in r_members:
    if votum['geschlecht'] == 'm':
        data_m.append(list(map(lambda tag: tag[0], votum['tags'])))
    else:
        data_w.append(list(map(lambda tag: tag[0], votum['tags'])))

# Build Dictionary
id2word_m = corpora.Dictionary(data_m)
id2word_w = corpora.Dictionary(data_w)

# Term Document Frequency
corpus_m = [id2word_m.doc2bow(text) for text in data_m]
corpus_w = [id2word_w.doc2bow(text) for text in data_w]

## Go LDA

In [32]:
lda_model_m = gensim.models.ldamodel.LdaModel(corpus=corpus_m, id2word=id2word_m, num_topics=20, random_state=100, update_every=1,
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [33]:
lda_model_w = gensim.models.ldamodel.LdaModel(corpus=corpus_w, id2word=id2word_w, num_topics=20, random_state=100, update_every=1,
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [34]:
root = Path('../export/models/2016-05-01/')

# Store LDA
lda_model_m.save(str(root / Path('lda_m.model')))
lda_model_w.save(str(root / Path('lda_w.model')))

# Store corpus & id2word
with open(root / Path('corpus_m'), 'w', encoding='utf-8') as f:
    f.write(json.dumps(corpus_m, ensure_ascii = False))
id2word_m.save(str(root / Path('id2word_m')))

with open(root / Path('corpus_w'), 'w', encoding='utf-8') as f:
    f.write(json.dumps(corpus_w, ensure_ascii = False))
id2word_w.save(str(root / Path('id2word_w')))

## Load models, if needed

In [10]:
root = Path('../export/models/2016-05-01/')

In [11]:
# If needed...
lda_model_m = gensim.models.LdaModel.load(str(root / Path('lda_m.model')))
lda_model_w = gensim.models.LdaModel.load(str(root / Path('lda_w.model')))

In [12]:
id2word_m = gensim.corpora.dictionary.Dictionary.load(str(root / 'id2word_m'))
id2word_w = gensim.corpora.dictionary.Dictionary.load(str(root / 'id2word_w'))

corpus_m = json.loads(open(root / Path('corpus_m'), 'r', encoding='utf-8').read())
corpus_w = json.loads(open(root / Path('corpus_w'), 'r', encoding='utf-8').read())

## Men

In [13]:
pyLDAvis.enable_notebook()

In [14]:
lda_viz = gensimvis.prepare(lda_model_m, corpus_m, id2word_m)
lda_viz

/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module i

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.409087  0.024930       1        1  37.844134
12    -0.353368 -0.269227       2        1  19.053769
11    -0.362633 -0.037848       3        1  17.438650
5     -0.318913  0.322195       4        1  15.001028
0     -0.009832  0.009499       5        1   2.651739
10    -0.020513 -0.075936       6        1   2.096596
19     0.057990  0.003390       7        1   1.282280
14     0.096993  0.004024       8        1   0.709790
7      0.093202  0.002527       9        1   0.685515
9      0.101183  0.002214      10        1   0.586451
4      0.094738  0.001167      11        1   0.522809
6      0.103951  0.002095      12        1   0.487227
15     0.108445  0.001888      13        1   0.444198
17     0.104396  0.002074      14        1   0.411245
13     0.115231  0.001507      15        1   0.278790
16     0.116725  0.001401      16        1   0.212420
1      0.117218  0.001364      17        1   0.196745
18     0.120153  0.001092      18        1   0.077342
8      0.121980  0.000837      19        1   0.014231
3      0.122141  0.000807      20        1   0.005043, topic_info=                Term          Freq         Total Category  logprob  loglift
50            Zürich  15358.000000  15358.000000  Default  30.0000  30.0000
82            Kanton  14415.000000  14415.000000  Default  29.0000  29.0000
174          Franken   6384.000000   6384.000000  Default  28.0000  28.0000
209        Millionen   4845.000000   4845.000000  Default  27.0000  27.0000
1685           Stadt   6691.000000   6691.000000  Default  26.0000  26.0000
...              ...           ...           ...      ...      ...      ...
24           konkret      0.000694    202.328202  Topic20 -11.5761  -2.6875
25             steht      0.000694   2971.017316  Topic20 -11.5761  -5.3743
26      unterstützen      0.000694   5686.550688  Topic20 -11.5761  -6.0235
27              voll      0.000694    216.744752  Topic20 -11.5761  -2.7564
29    übergeordneten      0.000694     37.102253  Topic20 -11.5761  -0.9913

[836 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
2214      2  0.999550          Absatz
132      11  0.980647           Abzug
1001     11  0.982078          Abzüge
2663     17  0.957113         Adresse
408       4  0.998764     Alternative
...     ...       ...             ...
74       14  0.989948         äussern
2512      4  0.999533    öffentlichen
4377      9  0.975627     überfordert
29       14  0.970291  übergeordneten
1621      3  0.999038       überzeugt

[681 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 13, 12, 6, 1, 11, 20, 15, 8, 10, 5, 7, 16, 18, 14, 17, 2, 19, 9, 4])

## Women

In [15]:
lda_viz = gensimvis.prepare(lda_model_w, corpus_w, id2word_w)
lda_viz

/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.379940 -0.122345       1        1  27.783472
7     -0.371533 -0.260654       2        1  22.437424
3     -0.353168  0.115369       3        1  20.172027
2     -0.345144  0.291505       4        1  19.987571
13     0.041067 -0.002012       5        1   2.116895
16     0.057677 -0.003492       6        1   1.631600
6      0.063268 -0.002891       7        1   1.388772
0      0.090161 -0.002130       8        1   0.794353
19     0.088077  0.000294       9        1   0.692435
18     0.089258 -0.002168      10        1   0.638564
10     0.097389 -0.004044      11        1   0.488100
5      0.090496 -0.001664      12        1   0.320779
12     0.103004 -0.001134      13        1   0.319044
17     0.102893 -0.001146      14        1   0.318720
1      0.101525 -0.000730      15        1   0.318038
8      0.103773 -0.000449      16        1   0.236412
15     0.104746 -0.000803      17        1   0.164413
9      0.105280 -0.000630      18        1   0.091150
11     0.105517 -0.000497      19        1   0.062024
14     0.105656 -0.000380      20        1   0.038208, topic_info=                Term         Freq        Total Category  logprob  loglift
66            Zürich  6661.000000  6661.000000  Default   30.000  30.0000
640          Franken  4178.000000  4178.000000  Default   29.000  29.0000
150        Gemeinden  3698.000000  3698.000000  Default   28.000  28.0000
20            Kanton  6499.000000  6499.000000  Default   27.000  27.0000
585            schon  3734.000000  3734.000000  Default   26.000  26.0000
...              ...          ...          ...      ...      ...      ...
5289    Tagesschulen     0.003669     0.871553  Topic20  -11.105   2.3995
8544          Limmat     0.003669     0.871513  Topic20  -11.105   2.3996
4998          Tunnel     0.003669     0.871564  Topic20  -11.105   2.3995
5001       Wipkingen     0.003669     0.871517  Topic20  -11.105   2.3996
5002  Wipkingerplatz     0.003669     0.871508  Topic20  -11.105   2.3996

[809 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
3186     11  0.995168                   ABG
228       6  0.992739                    AL
1515      1  0.925612                Absatz
1515     12  0.073665                Absatz
4745     12  0.946899               Absätze
...     ...       ...                   ...
2259      6  0.990890  überdurchschnittlich
1122      2  0.998735             überhaupt
955       3  0.997827            überweisen
3184     17  0.802066              üblichen
4392      6  0.992871               übrigen

[713 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 4, 3, 14, 17, 7, 1, 20, 19, 11, 6, 13, 18, 2, 9, 16, 10, 12, 15])

## 10 Years back

In [11]:
root = Path('../export/models/2011-05-01/')

lda_model_m = gensim.models.LdaModel.load(str(root / Path('lda_m.model')))
lda_model_w = gensim.models.LdaModel.load(str(root / Path('lda_w.model')))

id2word_m = gensim.corpora.dictionary.Dictionary.load(str(root / 'id2word_m'))
id2word_w = gensim.corpora.dictionary.Dictionary.load(str(root / 'id2word_w'))

corpus_m = json.loads(open(root / Path('corpus_m'), 'r', encoding='utf-8').read())
corpus_w = json.loads(open(root / Path('corpus_w'), 'r', encoding='utf-8').read())

### Men

In [12]:
lda_viz = gensimvis.prepare(lda_model_m, corpus_m, id2word_m)
lda_viz

/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.391652 -0.039819       1        1  36.867729
14     0.387118 -0.166442       2        1  22.265289
10     0.352430 -0.208281       3        1  16.515009
18     0.257214  0.314915       4        1  10.038485
3      0.171079  0.174966       5        1   4.904688
0      0.069210  0.085849       6        1   3.228511
19     0.031511  0.100566       7        1   2.050869
1     -0.092428 -0.018660       8        1   0.813151
9     -0.101523 -0.019044       9        1   0.689778
4     -0.110085 -0.021087      10        1   0.550120
5     -0.119711 -0.021758      11        1   0.532654
2     -0.122752 -0.021839      12        1   0.422717
7     -0.129272 -0.021758      13        1   0.339806
15    -0.130331 -0.021697      14        1   0.256282
8     -0.137721 -0.020950      15        1   0.150457
6     -0.140355 -0.020218      16        1   0.146009
17    -0.141895 -0.019730      17        1   0.095158
12    -0.143864 -0.018877      18        1   0.075474
13    -0.144417 -0.018575      19        1   0.053902
16    -0.145862 -0.017562      20        1   0.003910, topic_info=               Term         Freq         Total Category  logprob  loglift
174         Franken  14911.00000  14911.000000  Default  30.0000  30.0000
176       Gemeinden  11456.00000  11456.000000  Default  29.0000  29.0000
209       Millionen  11792.00000  11792.000000  Default  28.0000  28.0000
133          Antrag  14039.00000  14039.000000  Default  27.0000  27.0000
107              ja  21208.00000  21208.000000  Default  26.0000  26.0000
..              ...          ...           ...      ...      ...      ...
24          konkret      0.00071    931.595080  Topic20 -11.9573  -3.9375
25            steht      0.00071   6035.379002  Topic20 -11.9573  -5.8060
26     unterstützen      0.00071  10961.524680  Topic20 -11.9573  -6.4027
27             voll      0.00071    423.615179  Topic20 -11.9573  -3.1494
29   übergeordneten      0.00071    174.562030  Topic20 -11.9573  -2.2629

[856 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
3741      7  0.997164         Abgaben
2214      6  0.999541          Absatz
1947     17  0.986388          Ahnung
2981     15  0.976117            Alex
1152     14  0.946898       Amtsdauer
...     ...       ...             ...
404       7  0.999472        Änderung
405       7  0.999152      Änderungen
29        5  0.996780  übergeordneten
588       2  0.995159       überhaupt
588       3  0.004725       überhaupt

[677 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 15, 11, 19, 4, 1, 20, 2, 10, 5, 6, 3, 8, 16, 9, 7, 18, 13, 14, 17])

### Women

In [13]:
lda_viz = gensimvis.prepare(lda_model_w, corpus_w, id2word_w)
lda_viz

/Users/simon/Documents/projects/nzz_zh_kantonsrat_genderstudy/env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.453311  0.152971       1        1  64.175270
16    -0.360419  0.127951       2        1  12.789129
17    -0.154614 -0.237126       3        1   5.336812
6     -0.178108 -0.315255       4        1   5.243815
9      0.007533  0.001598       5        1   2.057662
13     0.012925 -0.003473       6        1   1.925252
11     0.035650  0.012270       7        1   1.626056
18     0.050197  0.016269       8        1   1.274136
4      0.049693  0.016090       9        1   1.181926
3      0.058713  0.018613      10        1   1.019246
12     0.081764  0.021537      11        1   0.728880
5      0.082646  0.021588      12        1   0.622910
1      0.087864  0.018546      13        1   0.447809
7      0.085034  0.023011      14        1   0.432610
15     0.098174  0.021366      15        1   0.315846
19     0.094570  0.021734      16        1   0.300637
8      0.093667  0.022030      17        1   0.222601
2      0.100665  0.020872      18        1   0.154224
0      0.102529  0.020300      19        1   0.121357
10     0.104829  0.019109      20        1   0.023820, topic_info=           Term         Freq        Total Category  logprob  loglift
640     Franken  5924.000000  5924.000000  Default  30.0000  30.0000
150   Gemeinden  5231.000000  5231.000000  Default  29.0000  29.0000
643   Millionen  4801.000000  4801.000000  Default  28.0000  28.0000
266     Prozent  4043.000000  4043.000000  Default  27.0000  27.0000
378       wurde  6495.000000  6495.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
6330      Strom     0.002857     0.979877  Topic20 -11.4426   2.5047
6311    Million     0.002857     0.979895  Topic20 -11.4426   2.5047
6536       KBIK     0.002857     0.980017  Topic20 -11.4426   2.5045
8433  Nachfrage     0.002857     0.979897  Topic20 -11.4426   2.5047
6212     vorher     0.002857     0.979895  Topic20 -11.4426   2.5047

[770 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
224       6  0.997030           1.
3186     10  0.996555          ABG
2885      8  0.990951  Abklärungen
1515      2  0.999251       Absatz
2675     19  0.833166       Abzüge
...     ...       ...          ...
4300      5  0.993321   überprüfen
955       6  0.997984   überweisen
1514      2  0.410650   überwiesen
1514      6  0.587127   überwiesen
743       6  0.994635   überzeugen

[565 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 17, 18, 7, 10, 14, 12, 19, 5, 4, 13, 6, 2, 8, 16, 20, 9, 3, 1, 11])